In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np


class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

# 70で作成した行列をロード
X_train = np.load("./matrix/x_train.npy")
Y_train = np.load("./matrix/y_train.npy")

X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor =torch.from_numpy(Y_train)
datasets = TextDataset(X_train, Y_train)
train_dataloader = DataLoader(datasets, shuffle=True, batch_size=64)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(300, 4),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.layer(x)
        return logits


# GPUが利用可能か確認
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = Model().to(device)
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

cuda


In [2]:
import time

batch_list = [1,2,4,8,16,32,64,128,256]

for batch_size in batch_list:
    train_dataloader = DataLoader(datasets, shuffle=True, batch_size=batch_size)

    start_time = time.time()

    size = len(datasets)
    epoch_loss = 0
    epoch_correct = 0
    total_samples = 0

    for batch, (X, y) in enumerate(train_dataloader):
        # 予測と損失の計算
        X = X.float().to(device)
        pred = model(X)
        y = y.to(device)
        loss = loss_fn(pred, y.to(torch.float64))

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 損失の累計
        epoch_loss += loss.item()

        # 正答数の累計
        predicted = pred.argmax(dim=1)
        correct = (predicted == y.argmax(dim=1)).sum().item()
        epoch_correct += correct
        total_samples += len(y)

    end_time = time.time()
    epoch_time = end_time - start_time
    # エポックごとの平均損失と正答率を計算
    avg_loss = epoch_loss / len(train_dataloader)
    avg_accuracy = epoch_correct / total_samples
    print(f"batch_size:{batch_size}, time:{epoch_time}, avg_loss:{avg_loss}, avg_accuracy:{avg_accuracy}")


/home/tajiri/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


batch_size:1, time:33.93031048774719, avg_loss:1.3150858393918383, avg_accuracy:0.7291725236622622
batch_size:2, time:16.31339430809021, avg_loss:1.2319985622431207, avg_accuracy:0.7800581013963077
batch_size:4, time:8.442572593688965, avg_loss:1.1999118395376835, avg_accuracy:0.7821197638459376
batch_size:8, time:3.910479784011841, avg_loss:1.1859054845875667, avg_accuracy:0.7813700684097086
batch_size:16, time:2.342961311340332, avg_loss:1.1793838176688272, avg_accuracy:0.7805266610439509
batch_size:32, time:1.166261911392212, avg_loss:1.17607502298701, avg_accuracy:0.7805266610439509
batch_size:64, time:0.3864326477050781, avg_loss:1.174764147319269, avg_accuracy:0.7807140849030082
batch_size:128, time:0.9898936748504639, avg_loss:1.17378723596611, avg_accuracy:0.7805266610439509
batch_size:256, time:0.5491130352020264, avg_loss:1.1734029126235088, avg_accuracy:0.7804329491144223
